In [ ]:
#reward function: SOFA score measures patient health (higher score/increase in score should be penalised)
#referenced from: Deep Reinforcement Learning for Sepsis Treatment
def calculate_reward(cur_score, next_score):
    reward = 0
    c0 = -0.025
    c1 = -0.125
    if cur_score == next_score and next_score > 0:
        reward += c0
    return reward + c1 * (next_score - cur_score)

#calculate SOFA score for each state
def calculate_sofa_score(MAP, urine, ALT, AST, PO2, lactic_acid, serum_creatinine, FiO2, GCS_total):
    cardiovascular_score = 0
    if MAP < 70:
        cardiovascular_score = 1
    
    respitory_score = 0
    if FiO2 > 0:
        p_f_ratio = PO2 / FiO2
        
        if p_f_ratio < 100:
            respitory_score = 4
        elif p_f_ratio < 200:
            respitory_score = 3
        elif p_f_ratio < 300:
            respitory_score = 2
        else:
            respitory_score = 1
    
    renal_score = 0
    if serum_creatinine >= 1.2 and serum_creatinine < 2:
        renal_score = 1
    elif serum_creatinine >= 2 and serum_creatinine < 3.5:
        renal_score = 2
    elif serum_creatinine >= 3.5 and serum_creatinine < 5:
        renal_score = 3
    elif serum_creatinine >= 5.0:
        renal_score = 4
    #divide daily urine output standards by 6 as the data is 4 hour interval
    if urine < 500.0 / 6:
        renal_score = max(renal_score, 3)
    if urine < 200.0 / 6:
        renal_score = max(renal_score, 4)

    #since bilirubin is not available, use ALT and AST to calculate liver score
    liver_score = 0
    if (ALT > 40 and AST > 40):
        liver_score = 1
    if (ALT > 120 or AST > 120):
        liver_score = 2
    if (ALT > 300 or AST > 300):
        liver_score = 3
    if (ALT > 1000 or AST > 1000):
        liver_score = 4

    neuro_score = 0
    if GCS_total < 6:
        neuro_score = 4
    elif GCS_total < 10:
        neuro_score = 3
    elif GCS_total < 13:
        neuro_score = 2
    elif GCS_total < 15:
        neuro_score = 1
    
    lactic_acid_score = 0
    if lactic_acid > 2:
        lactic_acid_score = 2
    
    return cardiovascular_score + respitory_score + renal_score + liver_score + neuro_score + lactic_acid_score


In [ ]:
df['SOFA_score'] = df.apply(lambda row: calculate_sofa_score(row['MAP'], row['urine'], row['ALT'], row['AST'], row['PO2'], row['lactic_acid'], row['serum_creatinine'], row['FiO2'], row['GCS_total']), axis=1)
